In [216]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
from textwrap import wrap
from pandas.api.types import CategoricalDtype
pd.set_option('display.float_format', '{:.2f}'.format)

nominal_mapping = {
    'Agree': 1,
    'Disagree': 0,
    "Unknown" : np.nan # Use numpy's NaN to represent missing values
}


In [3]:
business_df=pd.read_csv('CP201A Business Data.csv', delimiter = ',')
business_df.rename(columns={"Technology (list all with commas between)":"tech_use",
                        "Financial Stability": "financial_stability",
                        "Family Owned": "family_owned",
                        "Struggle without Technology": "tech_struggle",
                        "Person Surveyed": "respondent",
                        "Business Type": "type",
                        "Business Length Open": "length_open",
                        "Business ID" : "ID"}, inplace=True)

In [56]:
# Clean Categorical Data of Financial Stability
# financial_group: The financial stability of this business has fully recovered from the COVID-19 pandemic.

# Map 'Agree' and 'Disagree' groupings
group_mapping = {
    'Strongly Agree': 'Agree',
    'Somewhat Agree': 'Agree',
    'Strongly Disagree': 'Disagree',
    'Somewhat Disagree': 'Disagree',
    "Don't Know/NA" : 'Unknown'
}

# Create a new column 'housing_group' for the grouped categories
business_df['financial_group'] = business_df['financial_stability'].map(group_mapping)
# There are some NaNs, replace them with Unknown
business_df.fillna(value={"financial_group": "Unknown"}, inplace = True)

# Set dummy variables 
business_df['financialrecovery_dv'] = business_df['financial_group'].map(nominal_mapping)

In [229]:
# Clean up Business Type variable
business_type_mapping = {
    "Activity Based": "Other",
    "Healthcare/Education": "Other",
    "Services": "Retail and/or Services",
    "Retail + Services": "Retail and/or Services",
    "Retail": "Retail and/or Services",
    "Restaurant/Bar": "Restaurant/Bar",
    "Grocery/Market": "Grocery/Market",
    "Other":"Other"
}
business_df["type_group"] = business_df["type"].map(business_type_mapping)
business_df[["type_group"]].value_counts(dropna=False)

type_group            
Retail and/or Services    103
Restaurant/Bar             57
Grocery/Market             24
Other                      16
Name: count, dtype: int64

In [77]:

pd.crosstab(index=business_df['type_group'], columns=business_df['financial_group'], margins=True)

financial_group,Agree,Disagree,Unknown,All
type_group,,,,
Grocery/Market,9,10,5,24
Other,1,7,8,16
Restaurant/Bar,12,17,28,57
Retail and/or Services,34,42,27,103
All,56,76,68,200


In [6]:
# Business Length Open
# Create a new column for filtered length open that excludes 0-3 years
length_mapping = {
    "More than 10 years": "More than 10",
    "0-3 years": np.nan,
    "6-10 years": "4-10",
    "4-5 years": "4-10"
}
business_df['length_open_filtered'] = business_df['length_open'].map(length_mapping)
business_df['open_ten_plus_dv'] = business_df['length_open_filtered'].map({"More than 10":1,"4-10":0})
business_df['open_four_ten_dv'] = business_df['length_open_filtered'].map({"More than 10":0,"4-10":1}) 

In [232]:
# Neighborhood
pd.crosstab(index=business_df["Neighborhood"], columns="total", dropna=False)

col_0,total
Neighborhood,
Berkeley,55
Downtown Oakland,57
Fruitvale,24
Mission,28
SF Chinatown,36


## Charts

In [234]:
# Prep data for years open v. financial recovery
df_open = pd.DataFrame()
df_open["length_open"] = business_df['length_open_filtered']
df_open["financial_recovery"] = business_df['financial_group']
df_open["type"] = business_df['type_group']
df_open["neighborhood"] = business_df["Neighborhood"]

df_open = df_open.dropna(subset=["length_open"]) #drop the length_open NaNs rows

In [79]:
pd.crosstab(index=df_open['type'], columns=df_open['financial_recovery'], margins=True)

financial_recovery,Agree,Disagree,Unknown,All
type,,,,
Grocery/Market,8,9,1,18
Other,1,7,4,12
Restaurant/Bar,10,16,6,32
Retail and/or Services,29,41,6,76
All,48,73,17,138


In [242]:
#label cleanup for chart
length_labels = {
    "More than 10": "Before 2013",
    "4-10": "Between 2014-2019"
}
recovery_labels = {
    "Agree": "Fully Recovered",
    "Disagree": "Not Fully Recovered",
    "Unknown": "Unsure/Unknown"
}
df_open["length_open"] = df_open["length_open"].map(length_labels)
df_open["financial_recovery"] = df_open["financial_recovery"].map(recovery_labels)

In [85]:
financial_order = ['Fully Recovered', 'Not Fully Recovered', 'Unsure/Unknown']
alt.Chart(df_open).mark_bar(size=50).encode(
    x=alt.X('length_open').title('Year Business Location Opened').axis(labelAngle=0),
    y=alt.Y('count(financial_recovery)').stack('normalize').title('Percent of Respondents (N=138)'),
    color=alt.Color('financial_recovery', sort=financial_order).title('Financial Recovery Perception'),
    order = alt.Order('color_financial_recovery_sort_index:Q')
).properties(width=200)

alt.Chart(...)

In [222]:
# Data prep for financial recovery x business type
# Grouped bar needs percentage data already calculated
df_type_pct = df_open.copy()
df_type_pct = df_type_pct.groupby(["financial_recovery","type"]).size().reset_index(name='counts')
for t in df_type_pct["type"].unique():
    df_type_pct.loc[df_type_pct["type"] == t, "pct_type"] = df_type_pct["counts"] / df_type_pct.loc[df_type_pct["type"] == t, "counts"].sum()

In [228]:
type_order = ['Grocery/Market', 'Retail and/or Services', 'Restaurant/Bar', 'Other']
alt.Chart(df_type_pct).mark_bar().encode(
    x=alt.X('financial_recovery').title('').axis(labels=False,tickSize=0),
    y=alt.Y('pct_type').axis(format='%').scale(domain=(0,1)).title('Percent of Businesses (N=138)'),
    color=alt.Color('financial_recovery', sort=financial_order).title('Financial Recovery Perception'),
    order = alt.Order('color_financial_recovery_sort_index:Q'),
    column=alt.Column("type:N",sort=type_order, spacing=32, header=alt.Header(titleOrient='bottom', labelOrient='bottom')).title('')
).properties()

alt.Chart(...)

In [236]:
# Prep Data for Neighborhood
pd.crosstab(index=df_open["neighborhood"], columns=df_open['financial_recovery'], dropna=False)

financial_recovery,Agree,Disagree,Unknown
neighborhood,,,
Berkeley,18,18,4
Downtown Oakland,8,18,7
Fruitvale,8,10,2
Mission,5,14,3
SF Chinatown,9,13,1


In [244]:
# Prep Data for Neighborhood
# Grouped bar needs percentage data already calculated
df_hood_pct = df_open.copy()
df_hood_pct = df_hood_pct.groupby(["financial_recovery","neighborhood"]).size().reset_index(name='counts')
for t in df_hood_pct["neighborhood"].unique():
    df_hood_pct.loc[df_hood_pct["neighborhood"] == t, "pct_hood"] = df_hood_pct["counts"] / df_hood_pct.loc[df_hood_pct["neighborhood"] == t, "counts"].sum()

In [238]:
df_hood_pct

,financial_recovery,neighborhood,counts,pct_hood
0,Agree,Berkeley,18,0.45
1,Agree,Downtown Oakland,8,0.24
2,Agree,Fruitvale,8,0.40
3,Agree,Mission,5,0.23
4,Agree,SF Chinatown,9,0.39
5,Disagree,Berkeley,18,0.45
6,Disagree,Downtown Oakland,18,0.55
7,Disagree,Fruitvale,10,0.50
8,Disagree,Mission,14,0.64
9,Disagree,SF Chinatown,13,0.57


In [246]:
neighborhood_order = ['Berkeley', 'Fruitvale', 'SF Chinatown', 'Downtown Oakland', 'Mission']
alt.Chart(df_hood_pct).mark_bar().encode(
    x=alt.X('financial_recovery').title('').axis(labels=False,tickSize=0),
    y=alt.Y('pct_hood').axis(format='%').scale(domain=(0,1)).title('Percent of Businesses (N=138)'),
    color=alt.Color('financial_recovery', sort=financial_order).title('Financial Recovery Perception'),
    order = alt.Order('color_financial_recovery_sort_index:Q'),
    column=alt.Column("neighborhood:N",sort=neighborhood_order, spacing=30, header=alt.Header(titleOrient='bottom', labelOrient='bottom')).title('')
).properties()

alt.Chart(...)